In [ ]:
!apt-get update
!apt-get install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
# Colab 환경에서는 아래 셀을 먼저 실행하여 크롬과 크롬드라이버를 설치해야 합니다.
# !apt-get update
# !apt-get install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import time
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd

class Crawling:
    def __init__(self, query):
        self.query = query
        self.store_id = self.get_store_id()
        # Colab 환경에서 headless 모드 실행을 위한 크롬 옵션 설정
        self.chrome_options = Options()
        self.chrome_options.add_argument('--headless')
        self.chrome_options.add_argument('--no-sandbox')
        self.chrome_options.add_argument('--disable-dev-shm-usage')

    def get_store_id(self):
        url = f"https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query={self.query}"
        response = requests.get(url)
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        div_tag = soup.find('div', class_='LylZZ')
        if div_tag:
            a_tag = div_tag.find('a')
            if a_tag and a_tag.has_attr('href'):
                href = a_tag['href']
                match = re.search(r'place/(\d+)', href)
                if match:
                    return match.group(1)
        print("Store ID를 찾지 못했습니다.")
        return None

    def get_info(self):
        if self.store_id is None:
            print("Store ID가 없으므로 get_info()를 실행할 수 없습니다.")
            return None

        info_tabs = {
            "information": "T8RFa",
            "menu/list": "place_section_content",
            "feed": "place_section_content",
            "home": "PIbes",
            "booking": "place_section_content"
        }
        info_result = {}

        driver = webdriver.Chrome(options=self.chrome_options)
        for tab, class_name in info_tabs.items():
            url = f"https://pcmap.place.naver.com/restaurant/{self.store_id}/{tab}"
            driver.get(url)
            try:
                # 최대 10초 동안 해당 클래스의 요소가 로드되기를 대기
                wait = WebDriverWait(driver, 10)
                element = wait.until(EC.presence_of_element_located((By.CLASS_NAME, class_name)))
                content = element.text
            except (NoSuchElementException, TimeoutException):
                content = None
            info_result[tab] = content
        driver.quit()

        info_df = pd.DataFrame({key: [value] for key, value in info_result.items()})
        info_df = info_df.dropna(axis=1)
        return info_df

    def get_reviews(self):
        if self.store_id is None:
            print("Store ID가 없으므로 get_reviews()를 실행할 수 없습니다.")
            return None

        review_tab = "review/visitor"
        class_name = "pui__vn15t2"
        url = f"https://pcmap.place.naver.com/restaurant/{self.store_id}/{review_tab}"

        driver = webdriver.Chrome(options=self.chrome_options)
        driver.get(url)
        # 페이지 전체의 리뷰가 로드될 때까지 잠시 대기
        time.sleep(2)

        before_h = driver.execute_script("return window.scrollY")
        cnt = 0
        while cnt < 3:
            try:
                load_more_button = driver.find_element(By.XPATH, "//a[@class='fvwqf']")
                load_more_button.click()
                time.sleep(1)
            except Exception:
                pass  # 더보기 버튼이 없으면 넘어감

            driver.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.END)
            time.sleep(1)
            after_h = driver.execute_script("return window.scrollY")
            if after_h == before_h:
                break
            before_h = after_h
            cnt += 1

        # 리뷰 요소 추출
        content = driver.find_elements(By.CLASS_NAME, class_name)
        reviews = [element.text for element in content]
        driver.quit()

        review_df = pd.DataFrame(reviews, columns=['review'])
        return review_df

# 예시 실행 코드
if __name__ == "__main__":
    # 원하는 장소 이름을 query에 입력하세요.
    query = "돌담흑돼지 연동 본점"
    crawler = Crawling(query)

    print("가게 정보 수집 중...")
    info_df = crawler.get_info()
    if info_df is not None:
        print("가게 정보 (DataFrame):")
        print(info_df)

    print("\n리뷰 수집 중...")
    reviews_df = crawler.get_reviews()
    if reviews_df is not None:
        print("리뷰 (DataFrame):")
        print(reviews_df)
